In [1]:
reset -fs

In [2]:
import json
import pandas as pd
import requests
from collections import OrderedDict

searchman_key = ""

In [3]:
df_metadata = pd.read_csv("data/googleplay_metadata_phr.csv")
df_reviews = pd.read_csv("data/googleplay_reviews_phr.csv")


In [6]:
def fetch_reviews(app_id, api='searchman'):
    """Fetch metadata given an app_id
    """
    
    if api == 'searchman': 
        url = f"http://api.searchman.io/v1/android/us/app/reviews?appId={app_id}&apiKey={searchman_key}&count=100"
    r = requests.get(url)
    return json.loads(r.text)['data']

In [7]:
new_reviews_list = []
for ix, app in df_metadata.iterrows():
    print(app['name'])
    last_review_date = max(df_reviews[df_reviews['name'] == app['name']]['date'])
    new_reviews = fetch_reviews(app['id'])
    for i, review in enumerate(new_reviews):
        if review['timestamp'] > last_review_date:
            new_reviews_list.append({'name': app['name'], 'id': app['id'], 'title':review.get('title'),
                                     'author_name': review.get('author'),
                                     'author_uri': review.get('authorId'), 'rating':review.get('rating'),
                                     'date': review.get('timestamp'), 'voteSum': review.get('voteSum'),
                                     'voteCount': review.get('voteCount'), 'text': review.get('body')
                                    })
        else:
            break
new_reviews_list = [review for review in new_reviews_list if review['text'] is not None]

UPMC Health Plan
iBlueButton®
healow
Sharecare
Blue Shield of California
higi
myCigna
Medfusion Plus
Anthem Anywhere
FitnessSyncer
Teladoc
Aetna Mobile
Kaiser Permanente
MyHumana
Accolade, Inc.
FollowMyHealth®
Google Fit: Health and Activity Tracking
Medical records
My Medical
Medical Records App


In [9]:
print(f"{len(new_reviews_list)} new reviews.)
df_reviews = df_reviews.append(new_reviews_list, ignore_index=True)

In [12]:
df_reviews.tail()

,name,id,title,author_name,author_uri,rating,date,voteSum,voteCount,text
1747,Google Fit: Health and Activity Tracking,com.google.android.apps.fitness,None,Mohit G,107526919134556209809,1,2018-11-18 15:03:20,None,None,This app does not work property on my one plus...
1748,Google Fit: Health and Activity Tracking,com.google.android.apps.fitness,None,sagar suryawanshi,101956979010176828886,1,2018-11-18 14:54:32,None,None,I want to add leadboard How?
1749,Medical Records App,com.cliniconline,None,Abod Osama,115080537418668508998,4,2018-11-19 06:49:38,None,None,A great useful application but how can I trans...
1750,Medical Records App,com.cliniconline,None,Agnes Novita,110614731832618616830,5,2018-11-18 03:48:23,None,None,"Simple, easy, and useful"
1751,Medical Records App,com.cliniconline,None,Alfredo Baruc,116943082422817884463,5,2018-11-13 05:14:25,None,None,Very easy to use and is what I am looking for ...


In [13]:
df_reviews.to_csv("data/googleplay_reviews_phr.csv", index=False)